In [1]:
import numpy as np
from Utils.DTMC_Utils import DTMC_Utils
from Utils.GridGraph import GridGraph
from MultiRobotTargetSearch import MultiRobotTargetSearch
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from tqdm import tqdm

In [2]:
np.random.seed(12345)

In [3]:
reference_information_state = 1

In [ ]:
num_of_experiments = 1000
agent_numbers = [3, 5, 7, 9]
# grid_dimensions = [5, 8, 10, 12, 15, 20]
grid_dimensions = [5, 8, 10, 12]
alpha = 1

X, Y = np.meshgrid(agent_numbers, grid_dimensions)
Z = np.zeros_like(X, dtype=float)
Z_std = np.zeros_like(X, dtype=float)

for i, agent_number in tqdm(enumerate(agent_numbers), desc="Processing Agents", total=len(agent_numbers)):
    print(f"Processing agent: {agent_number}")
    
    for j, grid_dimension in tqdm(enumerate(grid_dimensions), desc=f"Grid Processing", leave=False, total=len(grid_dimensions)):
        nodesNumber = grid_dimension * grid_dimension
        grid_graph = GridGraph(grid_dimension, grid_dimension)
        consensus_times = []
        
        for k in range(num_of_experiments):
            agents = DTMC_Utils.initAgents(agent_number, nodesNumber)
            experiment = MultiRobotTargetSearch(agents, grid_graph, reference_information_state, None, alpha, _random_target=True, _preferred_direction=True)
            experiment.run()
            consensus_times.append(experiment.getMeanConsensusTime())
        
        Z[j, i] = np.mean(consensus_times)
        Z_std[j, i] = np.std(consensus_times)

Processing Agents:   0%|          | 0/4 [00:00<?, ?it/s]

Processing agent: 3



Grid Processing:  50%|█████     | 2/4 [07:00<08:11, 245.88s/it]

In [ ]:
output_file = 'preferred_direction_analysis.npz'

# Salva i dati come un file .npz, inclusi X e Y
np.savez(output_file, X=X, Y=Y, Z=Z, Z_std=Z_std, agent_numbers=agent_numbers, grid_dimensions=grid_dimensions)

print(f"Tutti i dati salvati correttamente in {output_file}")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')

cbar = fig.colorbar(surf, ax=ax, pad=0.15, aspect=3)

ax.set_xlabel('Agents number')
ax.set_ylabel('Grid dimension')
ax.set_zlabel('Average iterations until consensus')

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()

In [ ]:
# Media i valori di Z e Z_std per ogni numero di agenti (fisso X e varia Y)
Z_mean_over_grids = np.mean(Z, axis=0)
Z_std_over_grids = np.mean(Z_std, axis=0)

# Media i valori di Z e Z_std per ogni dimensione di griglia (fisso Y e varia X)
Z_mean_over_agents = np.mean(Z, axis=1)
Z_std_over_agents = np.mean(Z_std, axis=1)

# Plot con X (numero di agenti) e Y mediato
DTMC_Utils.plot_average_execution_time(
    agent_numbers, Z_mean_over_grids, Z_std_over_grids, 
    "Number of Agents", "Average Execution Time vs Number of Agents (averaged over grid sizes)"
)

# Plot con Y (dimensione della griglia) e X mediato
DTMC_Utils.plot_average_execution_time(
    grid_dimensions, Z_mean_over_agents, Z_std_over_agents, 
    "Grid Dimension", "Average Execution Time vs Grid Dimension (averaged over agent numbers)"
)